# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # Get non-repeated words
    vocab = set(text)
    
    # Create a dictionary {word: index}
    vocab_to_int = {word:i for i, word in enumerate(vocab)}
    
    # Create a dictionary {index: word}
    int_to_vocab = {i:word for i, word in enumerate(vocab)}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [9]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_lookup_table = {
        '.': "||period||",
        ',': "||comma||",
        '"': "||quotation_mark||",
        ';': "||semicolon||",
        '!': "||exclamation_mark||",
        '?': "||question_mark||",
        '(': "||left_parenthesis||",
        ')': "||right_parenthesis||",
        '-': "||dash||",
        '\n': "||return||",
    }        
    return token_lookup_table

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [12]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """

    X = []
    y = []
    i = 0
    while i < (len(words) - sequence_length):
        X.append(words[i:i+sequence_length])
        y.append(words[i+sequence_length])
        i += 1
    
    tensor_dataset = TensorDataset(torch.from_numpy(np.array(X)), torch.from_numpy(np.array(y)))
    dataloader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True)

    # return a dataloader
    return dataloader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [13]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 4,  5,  6,  7,  8],
        [18, 19, 20, 21, 22],
        [41, 42, 43, 44, 45],
        [30, 31, 32, 33, 34],
        [29, 30, 31, 32, 33],
        [24, 25, 26, 27, 28],
        [25, 26, 27, 28, 29],
        [10, 11, 12, 13, 14],
        [43, 44, 45, 46, 47],
        [19, 20, 21, 22, 23]])

torch.Size([10])
tensor([ 9, 23, 46, 35, 34, 29, 30, 15, 48, 24])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [17]:
import torch.nn as nn

"""
took from Sentiment_RNN_Solution.ipynb
"""
class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()

        # set class variables
        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out[-batch_size:]

        # return last sigmoid output and hidden state    
        return sig_out, hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''    
        # initialize hidden state with zero weights, and move to GPU if available

        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden
        
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [19]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    # From "Sentiment Prediction RNN - Training the Model"
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda(),

    # perform backpropagation and optimization
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])
            
    # zero accumulated gradients
    rnn.zero_grad()

    # get the output from the model
    output, h = rnn(inp, h)
    
    # calculate the loss and perform backprop
    loss, h = rnn.forward(inp, h)
    loss.sum().backward()
    loss_scalar = loss.sum().item()
    
    optimizer.step()
    
    # return the loss over a batch and the hidden state produced by our model
    return loss_scalar, h

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [21]:
# Data params
# Sequence Length
sequence_length = 15  # of words in a sequence
# Batch Size
batch_size = 64

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [22]:
# Training parameters
# Number of Epochs
num_epochs = 7
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 256
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 13572.942996887206

Epoch:    1/10    Loss: 104.41599149322509

Epoch:    1/10    Loss: 44.63458639907837

Epoch:    1/10    Loss: 24.6571106338501

Epoch:    1/10    Loss: 15.554544967651367

Epoch:    1/10    Loss: 10.300166083335876

Epoch:    1/10    Loss: 7.197774758338928

Epoch:    1/10    Loss: 5.28646186208725

Epoch:    1/10    Loss: 3.85770347738266

Epoch:    1/10    Loss: 2.8647155380249023

Epoch:    1/10    Loss: 2.152792395114899

Epoch:    1/10    Loss: 1.663551571726799

Epoch:    1/10    Loss: 1.2561545170545578

Epoch:    1/10    Loss: 0.9667574231624603

Epoch:    1/10    Loss: 0.743768217086792

Epoch:    1/10    Loss: 0.5831842755675316

Epoch:    1/10    Loss: 0.44344615799188614

Epoch:    1/10    Loss: 0.34626863452792167

Epoch:    1/10    Loss: 0.27035313001275063

Epoch:    1/10    Loss: 0.20757230792939663

Epoch:    1/10    Loss: 0.16264735496044158

Epoch:    1/10    Loss: 0.12659276224672794

Epoch:   

Epoch:    7/10    Loss: 1.8785584367186913e-08

Epoch:    7/10    Loss: 1.9700219841212175e-08

Epoch:    7/10    Loss: 1.9159217884201497e-08

Epoch:    7/10    Loss: 1.994126337168112e-08

Epoch:    7/10    Loss: 2.0280203427702757e-08

Epoch:    7/10    Loss: 1.8628249816465824e-08

Epoch:    7/10    Loss: 1.838435900269886e-08

Epoch:    7/10    Loss: 1.9461712627233397e-08

Epoch:    7/10    Loss: 1.8151211803818513e-08

Epoch:    7/10    Loss: 1.8080107755835684e-08

Epoch:    7/10    Loss: 1.68919359064823e-08

Epoch:    7/10    Loss: 1.7161849414915052e-08

Epoch:    7/10    Loss: 1.7621486115171337e-08

Epoch:    7/10    Loss: 1.968192011281289e-08

Epoch:    8/10    Loss: 1.7554748552934336e-08

Epoch:    8/10    Loss: 1.7486149160639285e-08

Epoch:    8/10    Loss: 1.9225575226933245e-08

Epoch:    8/10    Loss: 1.7955893832688475e-08

Epoch:    8/10    Loss: 1.713025877281282e-08

Epoch:    8/10    Loss: 1.7902521459056687e-08

Epoch:    8/10    Loss: 1.6731243480005985e-08

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pac

In [36]:
# as we se from the first run, 7 epochs do the job
num_epochs = 7

# Learning Rate
learning_rates = [0.01]
# Embedding Dimension
embedding_dims = [64,512]
# Hidden Dimension
hidden_dims = [64,512]
# Number of RNN Layers
n_layerss = [3,9]

for learning_rate in learning_rates:
    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            for n_layers in n_layerss:
                model_name = './trained_rnn'+'_'+str(embedding_dim)+'_'+str(hidden_dim)+'_'+str(n_layers)
                print("================================================================")
                print("learning_rate:", learning_rate)
                print("embedding_dim:", embedding_dim)
                print("hidden_dim:", hidden_dim)
                print("n_layers:", n_layers)
                print("model name:", model_name)
                print("================================================================")

                # create model and move to gpu if available
                rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
                if train_on_gpu:
                    rnn.cuda()

                # defining loss and optimization functions for training
                optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
                criterion = nn.CrossEntropyLoss()

                # training the model
                trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

                # saving the trained model
                helper.save_model(model_name, trained_rnn)
                print('Model Trained and Saved')

learning_rate: 0.01
embedding_dim: 64
hidden_dim: 64
n_layers: 3
model name: ./trained_rnn_64_64_3
Training for 7 epoch(s)...
Epoch:    1/7     Loss: 6780.579651462555

Epoch:    1/7     Loss: 18.099153697013854

Epoch:    1/7     Loss: 9.45455786037445

Epoch:    1/7     Loss: 5.5035782930850985

Epoch:    1/7     Loss: 3.6690689451694487

Epoch:    1/7     Loss: 2.6501901524066924

Epoch:    1/7     Loss: 1.8079136562943459

Epoch:    1/7     Loss: 1.3514124732613564

Epoch:    1/7     Loss: 0.9728663436770439

Epoch:    1/7     Loss: 0.7986327899694443

Epoch:    1/7     Loss: 0.6325625067055225

Epoch:    1/7     Loss: 0.4055070600509644

Epoch:    1/7     Loss: 0.40638503327965736

Epoch:    1/7     Loss: 0.28182210594415663

Epoch:    1/7     Loss: 0.2171906740926206

Epoch:    1/7     Loss: 0.18353585832193495

Epoch:    1/7     Loss: 0.1255821509324014

Epoch:    1/7     Loss: 0.09913737607374787

Epoch:    1/7     Loss: 0.08463309722952544

Epoch:    1/7     Loss: 0.0693651619

Epoch:    7/7     Loss: 4.9648789038917925e-09

Epoch:    7/7     Loss: 1.9826725928399053e-09

Epoch:    7/7     Loss: 2.0919560796673444e-08

Epoch:    7/7     Loss: 2.233350446472057e-09

Epoch:    7/7     Loss: 5.058321415916816e-09

Epoch:    7/7     Loss: 1.824125244443481e-08

Epoch:    7/7     Loss: 6.72474082433673e-09

Epoch:    7/7     Loss: 3.450931027439258e-08

Epoch:    7/7     Loss: 1.4007406850858087e-09

Epoch:    7/7     Loss: 1.399319385732425e-09

Epoch:    7/7     Loss: 1.7040808103212261e-09

Epoch:    7/7     Loss: 6.347107219424879e-09

Epoch:    7/7     Loss: 2.703987445860272e-09

Epoch:    7/7     Loss: 5.508614593906003e-09

Epoch:    7/7     Loss: 3.2697153047562955e-07

Epoch:    7/7     Loss: 8.074481586387485e-09

Epoch:    7/7     Loss: 1.2587709446378617e-09

Epoch:    7/7     Loss: 6.890939095986098e-10

Epoch:    7/7     Loss: 1.2029343927402984e-09

Model Trained and Saved
learning_rate: 0.01
embedding_dim: 64
hidden_dim: 64
n_layers: 9
model name:

Epoch:    6/7     Loss: 1.1919089492986798e-08

Epoch:    6/7     Loss: 1.1479253124851241e-08

Epoch:    6/7     Loss: 7.169319884674981e-09

Epoch:    6/7     Loss: 1.8488351903510797e-08

Epoch:    6/7     Loss: 5.9313010276779285e-08

Epoch:    6/7     Loss: 1.0008642848570806e-08

Epoch:    6/7     Loss: 7.241588792735034e-09

Epoch:    6/7     Loss: 7.1283143405683215e-09

Epoch:    6/7     Loss: 1.5741234950728005e-08

Epoch:    6/7     Loss: 1.1619264890029534e-08

Epoch:    6/7     Loss: 4.416994410209452e-08

Epoch:    7/7     Loss: 3.180277457067656e-08

Epoch:    7/7     Loss: 1.4546356238505403e-08

Epoch:    7/7     Loss: 1.5749394951529904e-08

Epoch:    7/7     Loss: 2.0258523008158234e-08

Epoch:    7/7     Loss: 4.128694090916491e-08

Epoch:    7/7     Loss: 1.9641767986868445e-08

Epoch:    7/7     Loss: 1.6602804670837002e-08

Epoch:    7/7     Loss: 2.105478479718693e-08

Epoch:    7/7     Loss: 4.346256688382384e-09

Epoch:    7/7     Loss: 1.7854277774743974e-07


Epoch:    5/7     Loss: 3.2518208024079342e-09

Epoch:    5/7     Loss: 3.961586433731678e-09

Epoch:    5/7     Loss: 3.706419882787415e-09

Epoch:    5/7     Loss: 3.6646248386951186e-09

Epoch:    5/7     Loss: 3.7547574814889014e-09

Epoch:    6/7     Loss: 3.3382094930351848e-09

Epoch:    6/7     Loss: 3.3054731560788754e-09

Epoch:    6/7     Loss: 3.6736563231998254e-09

Epoch:    6/7     Loss: 3.3340269642856414e-09

Epoch:    6/7     Loss: 2.947041423895236e-09

Epoch:    6/7     Loss: 3.1178565871314846e-09

Epoch:    6/7     Loss: 2.9834026990527284e-09

Epoch:    6/7     Loss: 3.2492355306334987e-09

Epoch:    6/7     Loss: 2.8583334195642073e-09

Epoch:    6/7     Loss: 3.024932958617388e-09

Epoch:    6/7     Loss: 3.255011704172972e-09

Epoch:    6/7     Loss: 2.5674090061711396e-09

Epoch:    6/7     Loss: 3.0945427704764938e-09

Epoch:    6/7     Loss: 2.7218069860723837e-09

Epoch:    6/7     Loss: 2.73238083436933e-09

Epoch:    6/7     Loss: 3.24791394840096e-09

E

Epoch:    5/7     Loss: 4.47056317176564e-09

Epoch:    5/7     Loss: 4.421604408610946e-09

Epoch:    5/7     Loss: 4.591725274893932e-09

Epoch:    5/7     Loss: 4.647186470640996e-09

Epoch:    5/7     Loss: 3.955616038830989e-09

Epoch:    5/7     Loss: 4.1083062459756905e-09

Epoch:    5/7     Loss: 4.088370631927241e-09

Epoch:    5/7     Loss: 3.919337273616819e-09

Epoch:    5/7     Loss: 3.977288289380709e-09

Epoch:    5/7     Loss: 3.9938696863028865e-09

Epoch:    5/7     Loss: 3.6927482487403296e-09

Epoch:    5/7     Loss: 3.88919137517263e-09

Epoch:    5/7     Loss: 3.9140085017486116e-09

Epoch:    5/7     Loss: 3.5652528549423578e-09

Epoch:    5/7     Loss: 3.664899914213038e-09

Epoch:    5/7     Loss: 3.6755995534987207e-09

Epoch:    5/7     Loss: 3.3491014064246373e-09

Epoch:    5/7     Loss: 3.672208366611862e-09

Epoch:    5/7     Loss: 3.3312853361389115e-09

Epoch:    5/7     Loss: 3.3275520402442637e-09

Epoch:    5/7     Loss: 3.404177903010286e-09

Epoch:

Epoch:    4/7     Loss: 3.055111438003688e-08

Epoch:    4/7     Loss: 4.818174921786289e-07

Epoch:    4/7     Loss: 6.130605766637743e-08

Epoch:    4/7     Loss: 1.2680278505767216e-07

Epoch:    4/7     Loss: 4.445623983551604e-08

Epoch:    4/7     Loss: 3.294181345145624e-08

Epoch:    4/7     Loss: 6.280290340995517e-08

Epoch:    4/7     Loss: 5.22572496019912e-08

Epoch:    4/7     Loss: 1.7141799469863606e-08

Epoch:    4/7     Loss: 3.581234096022238e-07

Epoch:    4/7     Loss: 1.9023064282410904e-07

Epoch:    4/7     Loss: 2.940661003360649e-07

Epoch:    4/7     Loss: 1.895938150748887e-07

Epoch:    4/7     Loss: 3.052621499485819e-08

Epoch:    4/7     Loss: 7.177529852629358e-09

Epoch:    4/7     Loss: 3.4869687358983836e-08

Epoch:    4/7     Loss: 1.0665832700883052e-08

Epoch:    4/7     Loss: 2.1144147027860222e-08

Epoch:    4/7     Loss: 1.905431506722677e-08

Epoch:    5/7     Loss: 3.510834927010429e-08

Epoch:    5/7     Loss: 8.361530303460001e-08

Epoch:  

Epoch:    3/7     Loss: 2.8881412162191226e-07

Epoch:    3/7     Loss: 2.9289407796945916e-07

Epoch:    3/7     Loss: 4.056174644785221e-07

Epoch:    3/7     Loss: 1.7973787062693404e-06

Epoch:    3/7     Loss: 3.534407808898077e-07

Epoch:    3/7     Loss: 3.899441972977759e-07

Epoch:    3/7     Loss: 3.945080386659333e-07

Epoch:    3/7     Loss: 4.7379930427591915e-08

Epoch:    3/7     Loss: 8.525576446120897e-08

Epoch:    3/7     Loss: 2.3334921617599973e-07

Epoch:    3/7     Loss: 4.351127980170531e-07

Epoch:    4/7     Loss: 1.2814043332969573e-07

Epoch:    4/7     Loss: 1.5334732370257154e-07

Epoch:    4/7     Loss: 1.086179100716735e-07

Epoch:    4/7     Loss: 1.7481590233091575e-07

Epoch:    4/7     Loss: 1.0454302734262244e-07

Epoch:    4/7     Loss: 7.363272644302738e-08

Epoch:    4/7     Loss: 6.110632741031936e-08

Epoch:    4/7     Loss: 2.2460316268450333e-07

Epoch:    4/7     Loss: 4.206346355648287e-08

Epoch:    4/7     Loss: 9.768516985954236e-08

Epo

Epoch:    2/7     Loss: 2.538475911137539e-06

Epoch:    2/7     Loss: 2.015300561197364e-06

Epoch:    2/7     Loss: 1.5458428957799697e-06

Epoch:    2/7     Loss: 1.210165543682251e-06

Epoch:    2/7     Loss: 9.681810212782694e-07

Epoch:    3/7     Loss: 7.189057095126027e-07

Epoch:    3/7     Loss: 4.794480361454134e-07

Epoch:    3/7     Loss: 3.712664777140162e-07

Epoch:    3/7     Loss: 3.2505715992670046e-07

Epoch:    3/7     Loss: 2.6685369284962236e-07

Epoch:    3/7     Loss: 2.031747857671462e-07

Epoch:    3/7     Loss: 1.578114788785001e-07

Epoch:    3/7     Loss: 1.387278426889793e-07

Epoch:    3/7     Loss: 1.0819362956127066e-07

Epoch:    3/7     Loss: 9.363431789211063e-08

Epoch:    3/7     Loss: 8.301489781636917e-08

Epoch:    3/7     Loss: 6.280840780092944e-08

Epoch:    3/7     Loss: 5.729815954502726e-08

Epoch:    3/7     Loss: 4.8142157389463594e-08

Epoch:    3/7     Loss: 4.186789796634116e-08

Epoch:    3/7     Loss: 4.0092571172323234e-08

Epoch: 

Epoch:    2/7     Loss: 0.0003489370390598197

Epoch:    2/7     Loss: 0.00027344926304067484

Epoch:    2/7     Loss: 0.00021939984445634763

Epoch:    2/7     Loss: 0.00017518620369082782

Epoch:    2/7     Loss: 0.00013803902965446469

Epoch:    2/7     Loss: 0.00010958398361981381

Epoch:    2/7     Loss: 8.625579373619985e-05

Epoch:    2/7     Loss: 6.640731572406366e-05

Epoch:    2/7     Loss: 5.218265831717872e-05

Epoch:    2/7     Loss: 4.177757001707505e-05

Epoch:    2/7     Loss: 3.300241828219441e-05

Epoch:    2/7     Loss: 2.571510840789415e-05

Epoch:    2/7     Loss: 1.988757537401398e-05

Epoch:    2/7     Loss: 1.5204691837425343e-05

Epoch:    2/7     Loss: 1.2280757517146412e-05

Epoch:    2/7     Loss: 9.227921382262138e-06

Epoch:    2/7     Loss: 7.316192433336255e-06

Epoch:    2/7     Loss: 5.8301970825596075e-06

Epoch:    2/7     Loss: 4.64406264381978e-06

Epoch:    2/7     Loss: 3.5566800534070355e-06

Epoch:    2/7     Loss: 2.842904416638703e-06

Epoch

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [24]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [26]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: uttered cleaning uttered uttered volcano vic: cleaning uttered cleaning apples cleaning volcano vic: volcano cleaning uttered volcano uttered uttered apples volcano vic: uttered uttered uttered volcano uttered apples vic: volcano uttered vic: uttered vic: apples cleaning volcano vic: uttered uttered vic: uttered apples apples cleaning apples volcano apples vic: vic: vic: uttered uttered cleaning uttered cleaning apples volcano vic: vic: cleaning apples volcano uttered uttered cleaning volcano vic: vic: volcano uttered volcano vic: apples vic: uttered volcano volcano vic: vic: apples uttered volcano vic: uttered uttered volcano uttered cleaning cleaning apples apples volcano apples vic: volcano cleaning vic: apples uttered uttered cleaning apples volcano apples cleaning vic: vic: uttered cleaning cleaning vic: apples uttered vic: apples cleaning cleaning apples cleaning vic: apples apples uttered uttered volcano volcano vic: apples volcano volcano cleaning cleaning cleaning apple

In [38]:
for learning_rate in learning_rates:
    for embedding_dim in embedding_dims:
        for hidden_dim in hidden_dims:
            for n_layers in n_layerss:
                print('')
                print('=======================================================================')
                print('')
                model_name = './trained_rnn'+'_'+str(embedding_dim)+'_'+str(hidden_dim)+'_'+str(n_layers)
                print(model_name)
                print('')
                trained_rnn = helper.load_model(model_name)
                pad_word = helper.SPECIAL_WORDS['PADDING']
                generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
                print(generated_script)
                print('')



./trained_rnn_64_64_3

jerry: volcano volcano apples vic: uttered apples cleaning apples uttered vic: vic: uttered apples uttered cleaning uttered vic: apples apples volcano uttered cleaning cleaning cleaning vic: vic: cleaning apples volcano volcano uttered apples volcano vic: volcano apples uttered volcano apples cleaning vic: uttered cleaning apples uttered uttered apples cleaning volcano apples uttered cleaning cleaning apples cleaning cleaning uttered cleaning cleaning volcano uttered volcano vic: cleaning cleaning apples cleaning apples uttered apples volcano apples cleaning apples vic: apples uttered cleaning uttered volcano volcano apples apples volcano apples volcano cleaning cleaning uttered uttered volcano uttered cleaning vic: cleaning volcano vic: volcano cleaning volcano cleaning vic: apples apples volcano apples vic: volcano apples vic: volcano volcano volcano vic: volcano apples cleaning cleaning vic: cleaning cleaning volcano apples apples apples apples vic: vic: vic

jerry: uttered cleaning volcano vic: cleaning uttered vic: cleaning cleaning cleaning volcano uttered vic: volcano vic: cleaning vic: apples vic: uttered vic: vic: volcano cleaning volcano volcano apples cleaning volcano apples apples volcano apples vic: volcano apples uttered vic: uttered vic: apples uttered vic: vic: vic: volcano cleaning cleaning vic: uttered uttered volcano uttered volcano cleaning cleaning cleaning volcano volcano volcano vic: uttered apples uttered uttered volcano cleaning cleaning volcano apples apples apples vic: vic: volcano vic: volcano cleaning vic: vic: uttered volcano vic: cleaning uttered vic: apples uttered volcano cleaning apples apples volcano uttered apples vic: apples uttered volcano volcano volcano uttered cleaning volcano apples vic: vic: cleaning uttered cleaning apples apples volcano vic: vic: cleaning volcano cleaning apples uttered cleaning uttered apples volcano uttered apples apples vic: apples volcano apples apples vic: volcano volcano vic: 

jerry: uttered volcano uttered apples uttered cleaning volcano cleaning vic: vic: volcano vic: vic: cleaning uttered apples cleaning apples uttered volcano volcano uttered cleaning volcano volcano volcano vic: apples uttered vic: cleaning volcano cleaning volcano volcano uttered cleaning vic: uttered volcano cleaning uttered volcano apples cleaning uttered cleaning apples cleaning cleaning cleaning cleaning vic: cleaning uttered cleaning uttered uttered uttered apples volcano cleaning uttered cleaning vic: cleaning apples cleaning cleaning vic: apples apples cleaning apples vic: vic: uttered apples vic: volcano vic: volcano vic: uttered uttered apples apples uttered uttered apples vic: uttered apples volcano volcano cleaning apples apples vic: apples volcano cleaning cleaning uttered volcano cleaning vic: apples vic: uttered uttered vic: apples volcano cleaning volcano cleaning uttered vic: apples apples volcano cleaning vic: vic: vic: cleaning apples vic: uttered apples vic: vic: volc

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [27]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.